    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


In [1]:
    import IPython, traitlets, jinja2, types, contextlib, textwrap, ast, argparse
    try:
        from .tangle import markdown_to_python
    except:
        from tangle import markdown_to_python

In [13]:
    def comparse_source(raw: str, source: str) -> bool: return list(
        map(str.rstrip, textwrap.dedent(raw).splitlines())
    ) == list(map(str.rstrip, source.splitlines()))

In [14]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        markdown = traitlets.Bool(True) 
        template = traitlets.Bool(True)
        format = traitlets.Bool(True)
        capture = traitlets.Bool(False)
        inline = traitlets.Bool(False)
        
        def run_cell(self, raw, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            source = markdown_to_python(raw)
            
            result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                self, source, store_history=store_history, 
                silent=silent, shell_futures=shell_futures)
            
            is_source =  comparse_source(raw, source)
            
            try:
                is_display = False
                if raw.strip() == ast.literal_eval(source).strip(): is_display=True
            except: ...
                
            if not (is_display or is_source):
                IPython.display.display(raw)
            return result

In [16]:
    original_methods = {}
    def load_ipython_extension(ip):
        ip.run_cell = types.MethodType(PidginShell.run_cell, ip)
        ip.enable_html_pager = True
    def unload_ipython_extension(ip):  
        ip.run_cell = types.MethodType(type(ip).run_cell, ip)

In [18]:
    parser = argparse.ArgumentParser(description="""Modify settings for a running pidgin kernel.""")

In [19]:
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())